In [1]:
import os
import numpy as np
from sklearn.cluster import SpectralClustering
from sklearn.metrics import normalized_mutual_info_score
import dgl
import torch

from collections import Counter

ModuleNotFoundError: No module named 'dgl'

In [ ]:
G = nx.read_gexf(f"../data/gexf/543.gexf", node_type=None, relabel=False, version='1.2draft')

In [ ]:
# group_id data for each node is missing,
# we use the color of each node to indicate their group.

"""
get the groud truth for this graph
"""
def truth(input_graph):
    group={}
    group_labels=dict(input_graph.nodes(data=True))

    res = []
    group_id = 0
    for k, v in group_labels.items():
        color = str(v['viz']['color'])
        if color not in group:
            group[color] = group_id
            res.append(group[color])
            group_id += 1
        else:
            res.append(group[color])
        
    return res

"""
get the total number of groups in the graph
"""

def total_groups(input_graph):
    return len(Counter(ground_truth(input_graph)))

In [ ]:
# Get the list of files and directories in the current directory
files = os.listdir("../data/gexf/")

# Split the file name and extension for each file and directory
files = [int(os.path.splitext(file)[0]) for file in files]

# Sort the list of file names without the extensions
sorted_files = sorted(files)

nmi_GN = []
nmi_SC = []
nmi_GNN = []
# Loop through every graph in the database.
for file in sorted_files:
    file_name = f'{file}.gexf'
    G = nx.read_gexf(f"../data/gexf/{file_name}", node_type=None, relabel=False, version='1.2draft')
    
    
    # Get the ground truth of this graph
    ground_truth = truth(G)
    
    """
    compute the communities using the Girvan-Newman algorithm, spectural clustering
    and graph neural network
    """
    
    """
    Girvan-Newman algorithm
    """
    communities_GN = nx.algorithms.community.centrality.girvan_newman(G)
    # get the first set of communities from the generator object
    predicted = tuple(sorted(c) for c in next(communities_GN))
    # flatten the list of sets into a list of node assignments
    predicted_flat_GN = [item for sublist in predicted for item in sublist]
    
    """
    Spectral Clustering
    """
    # create a SpectralClustering object and fit the data to it
    clustering = SpectralClustering(n_clusters=5)
    clustering.fit(X)
    
    

    """
    Graph Neural Network
    """
    
    # create a GNN with 3 hidden layers
    net = dgl.nn.GraphConv(4, 3, 3, 3)
    
    # initialize the GNN's parameters
    dgl.nn.init.xavier_uniform_(net.parameters())
    
    # forward pass through the GNN
    output = net(G)
    
    # compute the community assignments using the GNN's output
    community_assignments = torch.argmax(output, dim=1)

    # flatten the tensor of community assignments into a list
    predicted_flat_GNN = community_assignments.tolist()
    
    # compute the NMI score between the predicted and ground truth communities
    nmi_gn = normalized_mutual_info_score(ground_truth, predicted_flat_GN)
    nmi_sc= normalized_mutual_info_score(ground_truth, clustering.labels_)
    nmi_gnn = normalized_mutual_info_score(ground_truth, predicted_flat_GNN)
    
    # add NMI scores
    nmi_GN.append(nmi_gn)
    nmi_SC.append(nmi_sc)
    nmi_GNN.append(nmi_GNN)
    

In [ ]:
# plot graph vs nmi score

t = np.arange(0, 800)
ax = plt.axes()
ax.plot(t, nmi_GN, 'r', label='Girvan-Newman algorithm')
ax.plot(t, nmi_SC, 'g', label='Spectral Clustering')
ax.plot(t, nmi_GNN, 'b', label='Graph Neural Network')
ax.set_title('graph vs nmi score')
ax.set_xlabel('t')
ax.set_ylabel('A(t)')
ax.set_ylim(0, 20)
ax.legend()
plt.show()

In [ ]:
!pip install torch

     |█████████████▎                  | 368.9 MB 2.0 MB/s eta 0:04:25     |██████████                      | 276.7 MB 1.1 MB/s eta 0:09:30     |██████████                      | 280.6 MB 1.2 MB/s eta 0:08:45     |████████████                    | 336.2 MB 1.8 MB/s eta 0:05:08

In [ ]:
!pip install ngl